## This notebook will help you train a vanilla Point-Cloud AE with the basic architecture we used in our paper.
    (it assumes latent_3d_points is in the PYTHONPATH and the structural losses have been compiled)

In [1]:
import os
import os.path as osp
import sys

BASE = os.path.dirname(os.path.abspath(os.path.dirname("file"))) # latent_3D
sys.path.append(BASE)  # latent_3D
print(BASE)

from src.ae_templates import mlp_architecture_ala_iclr_18, default_train_params, mlp_architecture_ldgcnn
from src.autoencoder import Configuration as Conf
from src.point_net_ae import PointNetAutoEncoder, LDGCNNAutoEncoder

from src.in_out import snc_category_to_synth_id, create_dir, PointCloudDataSet, \
                                        load_all_point_clouds_under_folder

from src.tf_utils import reset_tf_graph
from src.general_utils import plot_3d_point_cloud

/home/sohee/coding/PointCloud/lgan/latent_3d_points
Instructions for updating:
Colocations handled automatically by placer.
Success


In [2]:
# # -*- coding: utf-8 -*-
# import numpy as np
# import os
# import os.path as osp
# import sys
# import matplotlib.pylab as plt

# BASE = os.path.dirname(os.path.abspath(os.path.dirname("file"))) # latent_3D
# sys.path.append(BASE) 

# from src.point_net_ae import PointNetAutoEncoder
# from src.autoencoder import Configuration as Conf
# from src.neural_net import MODEL_SAVER_ID

# from src.in_out import snc_category_to_synth_id, create_dir, PointCloudDataSet, \
#                                         load_all_point_clouds_under_folder

# from src.general_utils import plot_3d_point_cloud
# from src.tf_utils import reset_tf_graph

# from src.vanilla_gan import Vanilla_GAN
# from src.w_gan_gp import W_GAN_GP
# from src.generators_discriminators import latent_code_discriminator_two_layers,\
# latent_code_generator_two_layers


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

Define Basic Parameters

In [4]:
class_name = 'sofa'
ae_loss = 'chamfer'                   # Loss to optimize: 'emd' or 'chamfer'

top_out_dir = 'log/ldgcnn/'          # Use to save Neural-Net check-points etc.
top_in_dir = 'data/shape_net_core_uniform_samples_2048/' # Top-dir of where point-clouds are stored.

top_out_dir = osp.join(BASE, top_out_dir, class_name)
top_in_dir = osp.join(BASE, top_in_dir)


load_pre_trained_ae = False
#True #False
restore_epoch = 1000

experiment_name = 'single_class_ae_' + ae_loss 
n_pc_points = 2048                # Number of points per model.
bneck_size = 128                  # Bottleneck-AE size

#class_name = input('Give me the class name (e.g. "chair"): ').lower()

In [5]:
# train_params = default_train_params()
train_params = {'batch_size': 50, 
                'training_epochs': 500,
                'denoising': False,
                'learning_rate': 0.0005,
                'z_rotate': False,
                'saver_step': 10,
                'loss_display_step': 1
               }

Load Point-Clouds

In [6]:
syn_id = snc_category_to_synth_id()[class_name]
class_dir = osp.join(top_in_dir , syn_id)
all_pc_data = load_all_point_clouds_under_folder(class_dir, n_threads=8, file_ending='.ply', verbose=True)

3173 pclouds were loaded. They belong in 1 shape-classes.


Load default training parameters (some of which are listed beloq). For more details please print the configuration object.

    'batch_size': 50   
    
    'denoising': False     (# by default AE is not denoising)

    'learning_rate': 0.0005

    'z_rotate': False      (# randomly rotate models of each batch)
    
    'loss_display_step': 1 (# display loss at end of these many epochs)
    'saver_step': 10       (# over how many epochs to save neural-network)

In [7]:
encoder, decoder, enc_args, dec_args = mlp_architecture_ldgcnn(n_pc_points, bneck_size)
train_dir = create_dir(osp.join(top_out_dir, experiment_name))

In [8]:
conf = Conf(n_input = [n_pc_points, 3],
            loss = ae_loss,
            training_epochs = train_params['training_epochs'],
            batch_size = train_params['batch_size'],
            denoising = train_params['denoising'],
            learning_rate = train_params['learning_rate'],
            train_dir = train_dir,
            loss_display_step = train_params['loss_display_step'],
            saver_step = train_params['saver_step'],
            z_rotate = train_params['z_rotate'],
            encoder = encoder,
            decoder = decoder,
            encoder_args = enc_args,
            decoder_args = dec_args
           )
conf.experiment_name = experiment_name
conf.held_out_step = 5   # How often to evaluate/print out loss on 
                         # held_out data (if they are provided in ae.train() ).
conf.save(osp.join(train_dir, 'configuration'))

If you ran the above lines, you can reload a saved model like this:

In [10]:
if load_pre_trained_ae:
    conf = Conf.load(train_dir + '/configuration')
    reset_tf_graph()
    ae = LDGCNNAutoEncoder(conf.experiment_name, conf)
    ae.restore_model(conf.train_dir, epoch=restore_epoch)
else :
    reset_tf_graph()
    ae = LDGCNNAutoEncoder(conf.experiment_name, conf)

[autoreload of src.encoders_decoders failed: Traceback (most recent call last):
  File "/home/sohee/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/sohee/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 434, in superreload
    module = reload(module)
  File "/home/sohee/anaconda3/envs/python3/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/home/sohee/anaconda3/envs/python3/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 674, in exec_module
  File "<frozen importlib._bootstrap_external>", line 781, in get_code
  File "<frozen importlib._bootstrap_external>", line 741, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in

Building Encoder


Exception: filter length error: 1, only a length of 2 is supported.

Build AE Model.

In [ ]:
# reset_tf_graph()
# ae = PointNetAutoEncoder(conf.experiment_name, conf)

Train the AE (save output to train_stats.txt) 

In [ ]:
buf_size = 1 # Make 'training_stats' file to flush each output line regarding training.
fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)
train_stats = ae.train(all_pc_data, conf, log_file=fout)
fout.close()

In [ ]:
print(len(train_stats))

In [ ]:
plot_train_curve = True
import matplotlib.pylab as plt

print(conf.train_dir) # ../data/single_class_ae

if plot_train_curve:
    x = range(len(train_stats))
    loss = [t[1] for t in train_stats] # [0] epoch  [1] loss  [2] time

    plt.plot(x, loss)
    plt.title('AE training. (%s)' %(class_name))
    
    plt.tick_params(axis='x', which='both', bottom=False, top=False)
    plt.tick_params(axis='y', which='both', left=False, right=False)
    
    plt.xlabel('Epochs.') 
    plt.ylabel('Loss.')

In [ ]:
how_long_time = True


if how_long_time:
    time = [t[2] for t in train_stats] 
total = sum(time)  
m, s = divmod(total, 60)
h, m = divmod(m, 60)

print('Total runtime : %2d:%2d:%2d / %d epoch' %(h, m, s, len(time)))

Get a batch of reconstuctions and their latent-codes.

In [ ]:
# num = all_pc_data.num_examples
# print(num)

# feed_pc, feed_model_names, _ = all_pc_data.next_batch(num)
# reconstructions = ae.reconstruct(feed_pc)[0]
# latent_codes = ae.transform(feed_pc)

Use any plotting mechanism such as matplotlib to visualize the results.

In [ ]:
# i = 2
# plot_3d_point_cloud(reconstructions[i][:, 0], 
#                     reconstructions[i][:, 1], 
#                     reconstructions[i][:, 2], in_u_sphere=True);

# i = 4
# plot_3d_point_cloud(reconstructions[i][:, 0], 
#                     reconstructions[i][:, 1], 
#                     reconstructions[i][:, 2], in_u_sphere=True);

Save

In [ ]:
# import numpy as np
# import pandas as pd
# import os           
# import tensorflow as tf

# def createDir(dirname, path=os.getcwd()):
#     dirpath = osp.join(path, dirname)
#     try:
#         os.mkdir(dirpath)
#     except OSError as error:
#         print ("exist")
        
# # os.makedirs(osp.join(conf.train_dir, 'ae_npy'))
# # os.makedirs(osp.join(conf.train_dir, 'ae_txt'))

# createDir('ae_npy', conf.train_dir)
# createDir('ae_txt', conf.train_dir)

# # np.savez(osp.join(conf.train_dir, 'ae_npy', 'ae_reconstructions.npz'), reconstructions)
# np.save(osp.join(conf.train_dir, 'ae_npy', 'ae.npy'), reconstructions)      

In [ ]:
# f = open(osp.join(conf.train_dir, 'ae_reconstruction.txt'), 'a')
# f.write(reconstructions)
# f.close()

In [ ]:
    
# for i in range(0,num):
#     xx = reconstructions[i][:, 0]
#     x = xx.tolist()
#     #     x = tf.convert_to_tensor(xx, dtype = tf.float32)
    
#     yy = reconstructions[i][:, 1]
#     y = yy.tolist()
#     zz = reconstructions[i][:, 2]
#     z = zz.tolist()
    
#     np.save(osp.join(conf.train_dir, 'ae_npy', 'ae_recon_'+ str(i).zfill(5) +'.npy'), reconstructions[i])
    
    
      
#     p = open(osp.join(conf.train_dir, 'ae_txt', 'ae_recon_'+ str(i).zfill(5) +'.txt'), 'w+')
#     for k in range(0,2048):          
#         p.write('%8f\t%8f\t%8f\n' %(x[k], y[k], z[k]))
#     p.close()